In [ ]:
#This script scores and converts raw .log files from the EF version of the fractal n-back task to BIDS
#Inputs:
    #directory of raw task files (naming conversion bblid_scanid/bblid_scanid-frac2B_1.00_no1B2.log)
    #.xml file specific task to score trials 
#Outputs:
    #BIDS-valid events.tsv file
    #BIDS-valid json sidecar
    #Summary CSV with scoring information (including dprime) for all participants 

#adapted from Zizu

In [1]:
import glob
import json
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
from scipy.stats import norm
import math

In [2]:
#read template xml file, which is the key to scoring the task, change file path if needed 
#can be found on github, or on saturn at afp://saturn/Coordinators/Protocols/TED_PROTOCOLS/EXECUTIVE_829744/2022_data_freeze/inputs/
root = ET.parse('msmri522_2vs0_back.xml').getroot() 
scorelabel=root.getchildren()[5].getchildren() #the stimuli scores is in index 5 

/Users/krmurtha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
#set root_dir as directory where T1 task data is found -- file path will need to be updated, as this is MY local box sync
root_dir = '/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/SCANNER_TASK/'
#set task_dir to directory where individual subject folders are found
task_dir = 'T1_rename_2022_CORRECT'
all_files1=glob.glob(root_dir + task_dir + '*/*/*')
#print(all_files)

In [5]:
#set root_dir as directory where T2 task data is found
root_dir = '/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/SCANNER_TASK/'
#set task_dir to directory where individual subject folders are found
task_dir = 'SCANNER_TASK_T2'
all_files2=glob.glob(root_dir + task_dir + '*/*/*')

In [16]:
# TO CONVERT ALL FILES IN A DIRECTORY:
#read in each .log, convert to BIDS valid .tsv + accompanying .json
#bblid and scanid of each file will print out as they are converted 
for f in all_files1:
    try:
        if '.log' in f:
            string=f.split('/')
            string1=string[len(string)-2].split('_')
            bblid=string1[0]
            scanid=(string1[1])
            bb=pd.read_csv(f ,skiprows=3,sep='\t',header=None)
            bb.columns = ['Subject','Trial','EventType','Code','Time','TTime','Uncertainty0','Duration','Uncertainty1',
            'ReqTime','ReqDur','StimType','PairIndex']
            bb = bb[2:]
            bb[['Trial', 'Time', 'TTime', 'Duration', 'ReqTime', 'Uncertainty0', 'Uncertainty1', 'PairIndex']] = bb[['Trial', 'Time', 'TTime', 'Duration', 'ReqTime', 'Uncertainty0', 'Uncertainty1', 'PairIndex']].apply(pd.to_numeric)
            #print(bb)
            back0=[] #0back
            back2=[] #2back
            for i in range(0,len(scorelabel)):
                if scorelabel[i].get('category') == '0BACK':
                    back0.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])
                elif scorelabel[i].get('category') == '2BACK':
                    back2.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])
            scoresummary={'0BACK':back0,'2BACK':back2}
            c=list(scoresummary.items())

            allback=[]

            templateback0=c[0][1]
            templateback2=c[1][1]
            for i in range(0,len(templateback0)):
                a1=bb[bb['Trial'] >= np.int(templateback0[i][1])-2]
                a2=bb[bb['Trial'] <= np.int(templateback0[i][1])]
                aa=np.array(pd.merge(a1, a2,how='inner')['TTime'].to_list())
                if len(aa) > 6 :
                    if aa[0] > 0 :
                        response=aa[0]/10
                    else : 
                        res = next((i for i, j in enumerate(aa[range(0,len(aa),2)]) if j), None)
                        ste=res-1
                        centr=2*res-1
                        response=aa[centr]/10+ste*800
                else:
                    response=0
                allback.append([c[0][0],templateback0[i][1],templateback0[i][0],response])
            for i in range(0,len(templateback2)):
                a1=bb[bb['Trial'] >= np.int(templateback2[i][1])-2]
                a2=bb[bb['Trial'] <= np.int(templateback2[i][1])]
                aa=np.array(pd.merge(a1, a2,how='inner')['TTime'].to_list())
                if len(aa) > 6 :
                    if aa[0] > 0 :
                        response=aa[0]/10
                    else : 
                        res = next((i for i, j in enumerate(aa[range(0,len(aa),2)]) if j), None)
                        ste=res-1
                        centr=2*res-1
                        response=aa[centr]/10+ste*800
                else:
                    response=0

                allback.append([c[1][0],templateback2[i][1],templateback2[i][0],response])
            #now we convert into BIDS vaild headers!    
            #now we convert into BIDS vaild headers    
            dfallback=pd.DataFrame(allback)
            dfallback.columns=['task','Index','Results','ResponseTime']
            dfallback['Index'] = dfallback['Index'].astype(int)
            dfallback['Onset'] = 0.8 * dfallback['Index']
            dfallback['Duration'] = 3*0.8
            dfallback['ResponseTime'] = dfallback['ResponseTime'].astype(int)
            dfallback['Response_Time']=dfallback['ResponseTime']/1000
            dfallback=dfallback.drop(columns=['Index', 'ResponseTime'])
            dfallback=dfallback.rename(columns={'task': 'Trial_Type'})
            #next, score the task 
            score=[]
            for val in dfallback.values:
                if 'NR' in val and val[4]!=0:#no response expected, response detected
                    score.append('false_positive')
                if 'NR' in val and val[4]==0:#no response expected, no response detected
                    score.append('true_negative')
                if 'Match' in val and val[4]<=2.4 and val[4]>0:#response expected, response detected
                    score.append('true_positive')
                if 'Match' in val and (val[4]>2.4 or val[4]==0):#response expected, no response detected
                    score.append('false_negative')
            dfallback['Score']=score
            #display(dfallback)
            #and fill in the .json sidecar  
            sidecar={
                "Trial_type": {
                    "Description": "Indicator of type of action that is expected",
                    "Levels": {
                        "0BACK": "0back trial, participant responds to target picture",
                        "2BACK": "2back trial, participant responds if target picture is the same as picture shown 2 trials previously",
                    }
                }, 
                "Results":{
                    "Description": "Expected result of each trial",
                    "Levels":{
                        "NR": "No response, participant not expected to respond",
                        "Match":"Target is a match based on task rules, participant expected to repond",
                    }
                },

                "Onset": {
                    "Description": "Onset (in seconds) of the event measured from the beginning of the acquisition of the first volume in the corresponding task imaging data file",
                }, 

                "Duration":{
                    "Description":"Duration of the event (measured from onset) in seconds",
                },

                "Response_Time":{
                    "Description":"Time for a participant to make respond to each trial"
                },
                "Score":{
                    "Descripton":"Outcome of each trial.", 
                    "Levels":{
                        "false_positive":"no response expected, response detected",
                        "true_negative":"no response expected, no response detected",
                        "true_positive":"response expected, response detected",
                        "false_negative":"response expected, no response detected",
                    },
                },
            }
            print(bblid,scanid)
            #change file paths as necessary 
            dfallback.to_csv('/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/SCANNER_TASK/BIDS_2022/T1/sub-{0}_ses-{1}_task-fracback_acq-singleband_bold_events.tsv'.format(bblid, scanid), sep = '\t', index=False)
            with open('/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/SCANNER_TASK/BIDS_2022/T1/sub-{0}_ses-{1}_task-fracback_acq-singleband_bold_events.json'.format(bblid, scanid), 'w') as fp:
                json.dump(sidecar, fp)
    except:
        print("exception in {}".format(f))



20427 11233
21146 11488
21797 11862
21345 11702
20626 11324


In [ ]:
#TO CONVERT ONE FILE:
one_file=[]
for f in all_files1:
    if '99999' in f: ##sub in relevant scan ID for '99999'
        one_file.append(f)
#read in each .log, convert to BIDS valid .tsv + accompanying .json
#bblid and scanid of each file will print out as they are converted 
for f in one_file:
    try:
        if '.log' in f:
            string=f.split('/')
            string1=string[len(string)-2].split('_')
            bblid=string1[0]
            scanid=(string1[1])
            bb=pd.read_csv(f ,skiprows=3,sep='\t',header=None)
            bb.columns = ['Subject','Trial','EventType','Code','Time','TTime','Uncertainty0','Duration','Uncertainty1',
            'ReqTime','ReqDur','StimType','PairIndex']
            bb = bb[2:]
            bb[['Trial', 'Time', 'TTime', 'Duration', 'ReqTime', 'Uncertainty0', 'Uncertainty1', 'PairIndex']] = bb[['Trial', 'Time', 'TTime', 'Duration', 'ReqTime', 'Uncertainty0', 'Uncertainty1', 'PairIndex']].apply(pd.to_numeric)
            #print(bb)
            back0=[] #0back
            back2=[] #2back
            for i in range(0,len(scorelabel)):
                if scorelabel[i].get('category') == '0BACK':
                    back0.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])
                elif scorelabel[i].get('category') == '2BACK':
                    back2.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])
            scoresummary={'0BACK':back0,'2BACK':back2}
            c=list(scoresummary.items())

            allback=[]

            templateback0=c[0][1]
            templateback2=c[1][1]
            for i in range(0,len(templateback0)):
                a1=bb[bb['Trial'] >= np.int(templateback0[i][1])-2]
                a2=bb[bb['Trial'] <= np.int(templateback0[i][1])]
                aa=np.array(pd.merge(a1, a2,how='inner')['TTime'].to_list())
                if len(aa) > 6 :
                    if aa[0] > 0 :
                        response=aa[0]/10
                    else : 
                        res = next((i for i, j in enumerate(aa[range(0,len(aa),2)]) if j), None)
                        ste=res-1
                        centr=2*res-1
                        response=aa[centr]/10+ste*800
                else:
                    response=0
                allback.append([c[0][0],templateback0[i][1],templateback0[i][0],response])
            for i in range(0,len(templateback2)):
                a1=bb[bb['Trial'] >= np.int(templateback2[i][1])-2]
                a2=bb[bb['Trial'] <= np.int(templateback2[i][1])]
                aa=np.array(pd.merge(a1, a2,how='inner')['TTime'].to_list())
                if len(aa) > 6 :
                    if aa[0] > 0 :
                        response=aa[0]/10
                    else : 
                        res = next((i for i, j in enumerate(aa[range(0,len(aa),2)]) if j), None)
                        ste=res-1
                        centr=2*res-1
                        response=aa[centr]/10+ste*800
                else:
                    response=0

                allback.append([c[1][0],templateback2[i][1],templateback2[i][0],response])
            #now we convert into BIDS vaild headers!    
            #now we convert into BIDS vaild headers    
            dfallback=pd.DataFrame(allback)
            dfallback.columns=['task','Index','Results','ResponseTime']
            dfallback['Index'] = dfallback['Index'].astype(int)
            dfallback['Onset'] = 0.8 * dfallback['Index']
            dfallback['Duration'] = 3*0.8
            dfallback['ResponseTime'] = dfallback['ResponseTime'].astype(int)
            dfallback['Response_Time']=dfallback['ResponseTime']/1000
            dfallback=dfallback.drop(columns=['Index', 'ResponseTime'])
            dfallback=dfallback.rename(columns={'task': 'Trial_Type'})
            #next, score the task 
            score=[]
            for val in dfallback.values:
                if 'NR' in val and val[4]!=0:#no response expected, response detected
                    score.append('false_positive')
                if 'NR' in val and val[4]==0:#no response expected, no response detected
                    score.append('true_negative')
                if 'Match' in val and val[4]<=2.4 and val[4]>0:#response expected, response detected
                    score.append('true_positive')
                if 'Match' in val and (val[4]>2.4 or val[4]==0):#response expected, no response detected
                    score.append('false_negative')
            dfallback['Score']=score
            #display(dfallback)
            #and fill in the .json sidecar  
            sidecar={
                "Trial_type": {
                    "Description": "Indicator of type of action that is expected",
                    "Levels": {
                        "0BACK": "0back trial, participant responds to target picture",
                        "2BACK": "2back trial, participant responds if target picture is the same as picture shown 2 trials previously",
                    }
                }, 
                "Results":{
                    "Description": "Expected result of each trial",
                    "Levels":{
                        "NR": "No response, participant not expected to respond",
                        "Match":"Target is a match based on task rules, participant expected to repond",
                    }
                },

                "Onset": {
                    "Description": "Onset (in seconds) of the event measured from the beginning of the acquisition of the first volume in the corresponding task imaging data file",
                }, 

                "Duration":{
                    "Description":"Duration of the event (measured from onset) in seconds",
                },

                "Response_Time":{
                    "Description":"Time for a participant to make respond to each trial"
                },
                "Score":{
                    "Descripton":"Outcome of each trial.", 
                    "Levels":{
                        "false_positive":"no response expected, response detected",
                        "true_negative":"no response expected, no response detected",
                        "true_positive":"response expected, response detected",
                        "false_negative":"response expected, no response detected",
                    },
                },
            }
            print(bblid,scanid)
            #change file paths as necessary 
            dfallback.to_csv('/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/SCANNER_TASK/BIDS_2022/T2/sub-{0}_ses-{1}_task-fracback_acq-singleband_bold_events.tsv'.format(bblid, scanid), sep = '\t', index=False)
            with open('/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/SCANNER_TASK/BIDS_2022/T2/sub-{0}_ses-{1}_task-fracback_acq-singleband_bold_events.json'.format(bblid, scanid), 'w') as fp:
                json.dump(sidecar, fp)
    except:
        print("exception in {}".format(f))




In [21]:
#defines function to calculate dprime for all subs 
Z = norm.ppf

def SDT(hits, misses, fas, crs):
    """ returns d-prime measure given hits, misses, false alarms, and correct rejections"""
    # Floors an ceilings are replaced by half hits and half FA's
    half_hit = 0.5 / (hits + misses)
    half_fa = 0.5 / (fas + crs)
 
    # Calculate hit_rate and avoid d' infinity
    hit_rate = hits / (hits + misses)
    if hit_rate == 1: 
        hit_rate = 1 - half_hit
    if hit_rate == 0: 
        hit_rate = half_hit
 
    # Calculate false alarm rate and avoid d' infinity
    fa_rate = fas / (fas + crs)
    if fa_rate == 1: 
        fa_rate = 1 - half_fa
    if fa_rate == 0: 
        fa_rate = half_fa
 
    # Return d', beta, c and Ad'
    out = Z(hit_rate) - Z(fa_rate)
    
    return(out)

In [18]:
#set bids_dir to the directory where all BIDS valid csv's were saved to 
bids_dir = 'BIDS_2022/T1'
all_files_t1=glob.glob(root_dir + bids_dir + '*/*')
#print(all_files_2)


In [19]:
#set bids_dir to the directory where all BIDS valid csv's were saved to 
bids_dir = 'BIDS_2022/T2'
all_files_t2=glob.glob(root_dir + bids_dir + '*/*')
#print(all_files_2)


In [22]:
#loop through each .tsv and calculate summary scores for each trial outcome per condition
#ie: false pos, false neg, true pos, true neg for 0Back, 2Back, and combined (AllBack)
all_subs=[]
for f in all_files_t1:
    #initialize variables 
    back0fp=0
    back0fn=0
    back0tp=0
    back0tn=0
    back2fp=0
    back2fn=0
    back2tp=0
    back2tn=0
    if '.tsv' in f:
        string=f.split('/')
        file=(string[-1]).split('_')
        bblid=(file[0].split('-'))[-1]
        scanid=(file[1].split('-'))[-1]
        df=pd.read_csv(f ,sep='\t')
        #print(df)
        for val in df.values:
            if '0BACK' in val and 'false_positive' in val:
                back0fp+=1
            if '0BACK' in val and 'false_negative' in val:
                back0fn+=1
            if '0BACK' in val and 'true_positive' in val:
                back0tp+=1
            if '0BACK' in val and 'true_negative' in val:
                back0tn+=1
            if '2BACK' in val and 'false_positive' in val:
                back2fp+=1
            if '2BACK' in val and 'false_negative' in val:
                back2fn+=1
            if '2BACK' in val and 'true_positive' in val:
                back2tp+=1
            if '2BACK' in val and 'true_negative' in val:
                back2tn+=1
        summary={'bblid':bblid,'0BackFalsePositive':back0fp,'0BackFalseNegative':back0fn,'0BackTruePositive':back0tp,'0BackTrueNegative':back0tn,'2BackFalsePositive':back2fp,'2BackFalseNegative':back2fn,'2BackTruePositive':back2tp,'2BackTrueNegative':back2tn}
        summary.update({'AllBackTruePositive':(back0tp+back2tp), 'AllBackTrueNegative':(back0tn+back2tn),'AllBackFalsePositive':(back0fp+back2fp), 'AllBackFalseNegative':(back0fn+back2fn)})
        #add all correct + incorrect across conditions 
        summary.update({'0BackAllCorrect':(back0tp+back0tn),'0BackAllIncorrect':(back0fp+back0fn), '2BackAllCorrect':(back2tp+back2tn),'2BackAllIncorrect':(back2fp+back2fn),'AllBackAllCorrect':(back2tp+back2tn+back0tp+back0tn),'AllBackAllIncorrect':(back0fp+back0fn+back2fp+back2fn)})
        #add hit/false alarm rate across conditions (ie: true pos/ # targets, false pos/ # foils)
        summary.update({'0BackHitRate':(back0tp/15),'0BackFalseAlarmRate':(back0fp/45),'2BackHitRate':(back2tp/15),'2BackFalseAlarmRate':(back2fp/45),'AllBackHitRate':((back0tp+back2tp)/30),'AllBackFalseAlarmRate':((back0fp+back2fp)/90)})
        #calc dprime for various conditions using function defined above
        back0d=SDT(back0tp, back0fn, back0fp, back0tn)
        back2d=SDT(back2tp, back2fn, back2fp, back2tn)
        backAllD=SDT((back0tp+back2tp), (back0fn+back2fn), (back0fp+back2fp), (back0tn+back2tn))
        #add to dictionary  
        summary.update({'0BackDprime':back0d, '2BackDprime':back2d, 'AllBackDprime':backAllD})
        all_subs.append(summary)
        #print(all_subs)
        all_subs_df=pd.DataFrame(all_subs)
display(all_subs_df)
#change file paths as necessary
#all_subs_df.to_csv('/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/summary.csv', sep = ',', index=False)




,0BackAllCorrect,0BackAllIncorrect,0BackDprime,0BackFalseAlarmRate,0BackFalseNegative,0BackFalsePositive,0BackHitRate,0BackTrueNegative,0BackTruePositive,2BackAllCorrect,...,AllBackAllCorrect,AllBackAllIncorrect,AllBackDprime,AllBackFalseAlarmRate,AllBackFalseNegative,AllBackFalsePositive,AllBackHitRate,AllBackTrueNegative,AllBackTruePositive,bblid
0,56,4,3.181544,0.088889,0,4,1.000000,41,15,48,...,104,16,2.009465,0.100000,7,9,0.766667,81,23,21493
1,58,2,3.535203,0.044444,0,2,1.000000,43,15,49,...,107,13,2.225689,0.044444,9,4,0.700000,86,21,21554
2,59,1,3.843789,0.022222,0,1,1.000000,44,15,49,...,108,12,2.721726,0.111111,2,10,0.933333,80,28,21325
3,60,0,4.120463,0.000000,0,0,1.000000,45,15,51,...,111,9,2.782638,0.066667,3,6,0.900000,84,27,20918
4,53,7,2.189250,0.088889,3,4,0.800000,41,12,47,...,100,20,1.584738,0.066667,14,6,0.533333,84,16,21580
5,60,0,4.120463,0.000000,0,0,1.000000,45,15,49,...,109,11,2.468508,0.066667,5,6,0.833333,84,25,20812
6,57,3,3.335001,0.066667,0,3,1.000000,42,15,52,...,109,11,3.054555,0.111111,1,10,0.966667,80,29,21712
7,60,0,4.120463,0.000000,0,0,1.000000,45,15,50,...,110,10,2.561828,0.033333,7,3,0.766667,87,23,21738
8,57,3,3.335001,0.066667,0,3,1.000000,42,15,54,...,111,9,2.782638,0.066667,3,6,0.900000,84,27,21224
9,45,15,0.452633,0.000000,15,0,0.000000,45,0,45,...,90,30,0.411140,0.000000,30,0,0.000000,90,0,20865


In [23]:
#and for T2
all_subs=[]
for f in all_files_t2:
    #initialize variables 
    back0fp=0
    back0fn=0
    back0tp=0
    back0tn=0
    back2fp=0
    back2fn=0
    back2tp=0
    back2tn=0
    if '.tsv' in f:
        string=f.split('/')
        file=(string[-1]).split('_')
        bblid=(file[0].split('-'))[-1]
        scanid=(file[1].split('-'))[-1]
        df=pd.read_csv(f ,sep='\t')
        #print(df)
        for val in df.values:
            if '0BACK' in val and 'false_positive' in val:
                back0fp+=1
            if '0BACK' in val and 'false_negative' in val:
                back0fn+=1
            if '0BACK' in val and 'true_positive' in val:
                back0tp+=1
            if '0BACK' in val and 'true_negative' in val:
                back0tn+=1
            if '2BACK' in val and 'false_positive' in val:
                back2fp+=1
            if '2BACK' in val and 'false_negative' in val:
                back2fn+=1
            if '2BACK' in val and 'true_positive' in val:
                back2tp+=1
            if '2BACK' in val and 'true_negative' in val:
                back2tn+=1
        summary_2={'bblid':bblid,'0BackFalsePositive':back0fp,'0BackFalseNegative':back0fn,'0BackTruePositive':back0tp,'0BackTrueNegative':back0tn,'2BackFalsePositive':back2fp,'2BackFalseNegative':back2fn,'2BackTruePositive':back2tp,'2BackTrueNegative':back2tn}
        summary_2.update({'AllBackTruePositive':(back0tp+back2tp), 'AllBackTrueNegative':(back0tn+back2tn),'AllBackFalsePositive':(back0fp+back2fp), 'AllBackFalseNegative':(back0fn+back2fn)})
        #add all correct + incorrect across conditions 
        summary_2.update({'0BackAllCorrect':(back0tp+back0tn),'0BackAllIncorrect':(back0fp+back0fn), '2BackAllCorrect':(back2tp+back2tn),'2BackAllIncorrect':(back2fp+back2fn),'AllBackAllCorrect':(back2tp+back2tn+back0tp+back0tn),'AllBackAllIncorrect':(back0fp+back0fn+back2fp+back2fn)})
        #add hit/false alarm rate across conditions (ie: true pos/ # targets, false pos/ # foils)
        summary_2.update({'0BackHitRate':(back0tp/15),'0BackFalseAlarmRate':(back0fp/45),'2BackHitRate':(back2tp/15),'2BackFalseAlarmRate':(back2fp/45),'AllBackHitRate':((back0tp+back2tp)/30),'AllBackFalseAlarmRate':((back0fp+back2fp)/90)})
        #calc dprime for various conditions using function defined above
        back0d=SDT(back0tp, back0fn, back0fp, back0tn)
        back2d=SDT(back2tp, back2fn, back2fp, back2tn)
        backAllD=SDT((back0tp+back2tp), (back0fn+back2fn), (back0fp+back2fp), (back0tn+back2tn))
        #add to dictionary  
        summary_2.update({'0BackDprime':back0d, '2BackDprime':back2d, 'AllBackDprime':backAllD})
        all_subs.append(summary_2)
        #print(all_subs)
        all_subs_df=pd.DataFrame(all_subs)
display(all_subs_df)
#change file paths as necessary
#all_subs_df.to_csv('/Users/krmurtha/Box Sync/EXECUTIVE_FUNCTION/summary.csv', sep = ',', index=False)



,0BackAllCorrect,0BackAllIncorrect,0BackDprime,0BackFalseAlarmRate,0BackFalseNegative,0BackFalsePositive,0BackHitRate,0BackTrueNegative,0BackTruePositive,2BackAllCorrect,...,AllBackAllCorrect,AllBackAllIncorrect,AllBackDprime,AllBackFalseAlarmRate,AllBackFalseNegative,AllBackFalsePositive,AllBackHitRate,AllBackTrueNegative,AllBackTruePositive,bblid
0,59,1,3.843789,0.022222,0,1,1.000000,44,15,58,...,117,3,3.843789,0.022222,1,2,0.966667,88,29,20188
1,58,2,3.535203,0.044444,0,2,1.000000,43,15,51,...,109,11,2.468508,0.066667,5,6,0.833333,84,25,21023
2,60,0,4.120463,0.000000,0,0,1.000000,45,15,47,...,107,13,2.315050,0.088889,5,8,0.833333,82,25,20572
3,60,0,4.120463,0.000000,0,0,1.000000,45,15,50,...,110,10,2.810948,0.011111,9,1,0.700000,89,21,20916
4,60,0,4.120463,0.000000,0,0,1.000000,45,15,59,...,119,1,4.373099,0.000000,1,0,0.966667,90,29,19861
5,60,0,4.120463,0.000000,0,0,1.000000,45,15,53,...,113,7,2.982840,0.044444,3,4,0.900000,86,27,20712
6,55,5,2.542909,0.044444,3,2,0.800000,43,12,52,...,107,13,2.228999,0.066667,7,6,0.766667,84,23,21161
7,60,0,4.120463,0.000000,0,0,1.000000,45,15,51,...,111,9,2.668710,0.044444,5,4,0.833333,86,25,21104
8,57,3,3.335001,0.066667,0,3,1.000000,42,15,41,...,98,22,2.084123,0.211111,3,19,0.900000,71,27,20936
9,60,0,4.120463,0.000000,0,0,1.000000,45,15,49,...,109,11,2.717275,0.011111,10,1,0.666667,89,20,20310
